In [54]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
from pyspark.sql.functions import isnan, when, count, col
spark = SparkSession.builder.appName("my-app-name").getOrCreate()
from pyspark.sql.functions import udf

In [2]:
df_2019_07 = spark.read.parquet("s3://jw1987/fhv_tripdata_2019-07.parquet")
df_2019_08 = spark.read.parquet("s3://jw1987/fhv_tripdata_2019-08.parquet")

In [34]:
df_2019_07.printSchema()

root
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropOff_datetime: timestamp (nullable = true)
 |-- PUlocationID: double (nullable = true)
 |-- DOlocationID: double (nullable = true)



In [7]:
df_2019_07.count()

1947743

In [8]:
df_2019_08.count()

1880408

In [3]:
df_2019_07.show(5)

+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|dispatching_base_num|    pickup_datetime|   dropOff_datetime|PUlocationID|DOlocationID|SR_Flag|Affiliated_base_number|
+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|              B00013|2019-07-01 00:40:00|2019-07-01 01:19:25|       264.0|       264.0|   null|                B00013|
|              B00014|2019-07-01 00:00:00|2019-07-01 01:06:35|       264.0|       264.0|   null|                B00014|
|              B00014|2019-07-01 00:04:27|2019-07-01 00:07:38|       264.0|       264.0|   null|                B00014|
|              B00014|2019-07-01 00:39:51|2019-07-01 00:42:24|       264.0|       264.0|   null|                B00014|
|              B00014|2019-07-01 00:10:00|2019-07-01 02:49:25|       264.0|       264.0|   null|                B00014|
+--------------------+------------------

In [6]:
df_2019_07.select([count(when(col(c).isNull(), c)).alias(c) for c in df_2019_07.columns]).show()

+--------------------+---------------+----------------+------------+------------+-------+----------------------+
|dispatching_base_num|pickup_datetime|dropOff_datetime|PUlocationID|DOlocationID|SR_Flag|Affiliated_base_number|
+--------------------+---------------+----------------+------------+------------+-------+----------------------+
|                   0|              0|               0|       11055|         164|1947743|                 15526|
+--------------------+---------------+----------------+------------+------------+-------+----------------------+



In [9]:
df_2019_08.select([count(when(col(c).isNull(), c)).alias(c) for c in df_2019_08.columns]).show()

+--------------------+---------------+----------------+------------+------------+-------+----------------------+
|dispatching_base_num|pickup_datetime|dropOff_datetime|PUlocationID|DOlocationID|SR_Flag|Affiliated_base_number|
+--------------------+---------------+----------------+------------+------------+-------+----------------------+
|                   0|              0|               0|       56027|          51|1880408|                 12178|
+--------------------+---------------+----------------+------------+------------+-------+----------------------+



In [27]:
df_2019_07 = df_2019_07.filter(~df_2019_07.PUlocationID.isNull())

In [28]:
df_2019_08 = df_2019_08.filter(~df_2019_08.PUlocationID.isNull())

In [29]:
df_2019_07 = df_2019_07.select("pickup_datetime", "dropOff_datetime", "PUlocationID", "DOlocationID")

In [30]:
df_2019_08 = df_2019_08.select("pickup_datetime", "dropOff_datetime", "PUlocationID", "DOlocationID")

In [24]:
df_2019_07.show(5)

+-------------------+-------------------+------------+------------+
|    pickup_datetime|   dropOff_datetime|PUlocationID|DOlocationID|
+-------------------+-------------------+------------+------------+
|2019-07-01 00:40:00|2019-07-01 01:19:25|       264.0|       264.0|
|2019-07-01 00:00:00|2019-07-01 01:06:35|       264.0|       264.0|
|2019-07-01 00:04:27|2019-07-01 00:07:38|       264.0|       264.0|
|2019-07-01 00:39:51|2019-07-01 00:42:24|       264.0|       264.0|
|2019-07-01 00:10:00|2019-07-01 02:49:25|       264.0|       264.0|
+-------------------+-------------------+------------+------------+
only showing top 5 rows



In [31]:
df_2019_07.count()

1936688

In [32]:
df_2019_08.count()

1824381

# adjuct the time

In [41]:
from pyspark.sql import functions as f
df_2019_07 = df_2019_07.withColumn("pickup_datetime", f.from_unixtime(f.unix_timestamp(df_2019_07.pickup_datetime), "yyyy-MM-dd HH:MM:SS"))

In [42]:
df_2019_08 = df_2019_08.withColumn("pickup_datetime", f.from_unixtime(f.unix_timestamp(df_2019_08.pickup_datetime), "yyyy-MM-dd HH:MM:SS"))

In [43]:
df_2019_08.printSchema()

root
 |-- pickup_datetime: string (nullable = true)
 |-- dropOff_datetime: timestamp (nullable = true)
 |-- PUlocationID: double (nullable = true)
 |-- DOlocationID: double (nullable = true)



In [55]:
def adj_time(string):
    return string[:13] + ":00:00"

In [56]:
adj_time_UDF = udf(lambda m: adj_time(m))

In [60]:
df_2019_08 = df_2019_08.withColumn("pickup_timeslot",adj_time_UDF(col("pickup_datetime")))

In [61]:
df_2019_07 = df_2019_07.withColumn("pickup_timeslot",adj_time_UDF(col("pickup_datetime")))

In [62]:
df_2019_08.show(5)

+-------------------+-------------------+------------+------------+-------------------+
|    pickup_datetime|   dropOff_datetime|PUlocationID|DOlocationID|    pickup_timeslot|
+-------------------+-------------------+------------+------------+-------------------+
|2019-08-01 00:08:00|2019-08-01 00:54:00|       264.0|       264.0|2019-08-01 00:00:00|
|2019-08-01 00:08:00|2019-08-01 00:33:00|       264.0|       264.0|2019-08-01 00:00:00|
|2019-08-01 00:08:00|2019-08-01 01:34:33|       264.0|       264.0|2019-08-01 00:00:00|
|2019-08-01 00:08:00|2019-08-01 01:55:25|       264.0|       264.0|2019-08-01 00:00:00|
|2019-08-01 00:08:00|2019-08-01 03:48:19|       264.0|       264.0|2019-08-01 00:00:00|
+-------------------+-------------------+------------+------------+-------------------+
only showing top 5 rows



In [64]:
dfcount_2019_07 = df_2019_07.groupBy("pickup_timeslot", "PUlocationID").count()

In [65]:
dfcount_2019_07.count()

87512

In [66]:
dfcount_2019_08 = df_2019_08.groupBy("pickup_timeslot", "PUlocationID").count()

In [68]:
dfcount_2019_08.show(5)

+-------------------+------------+-----+
|    pickup_timeslot|PUlocationID|count|
+-------------------+------------+-----+
|2019-08-02 23:00:00|       264.0| 1634|
|2019-08-18 03:00:00|       129.0|    4|
|2019-08-01 00:00:00|       186.0|    2|
|2019-08-01 03:00:00|       198.0|    7|
|2019-08-01 12:00:00|        64.0|    3|
+-------------------+------------+-----+
only showing top 5 rows



In [67]:
dfcount_2019_08.count()

95344

In [71]:
dfcount_2019_08.sort(col("count").desc()).show()

+-------------------+------------+-----+
|    pickup_timeslot|PUlocationID|count|
+-------------------+------------+-----+
|2019-08-01 09:00:00|       264.0| 3846|
|2019-08-07 09:00:00|       264.0| 3812|
|2019-08-02 09:00:00|       264.0| 3801|
|2019-08-08 08:00:00|       264.0| 3799|
|2019-08-15 09:00:00|       264.0| 3752|
|2019-08-08 09:00:00|       264.0| 3749|
|2019-08-09 09:00:00|       264.0| 3734|
|2019-08-23 09:00:00|       264.0| 3726|
|2019-08-13 08:00:00|       264.0| 3715|
|2019-08-02 08:00:00|       264.0| 3708|
|2019-08-14 09:00:00|       264.0| 3706|
|2019-08-05 08:00:00|       264.0| 3702|
|2019-08-01 08:00:00|       264.0| 3690|
|2019-08-12 09:00:00|       264.0| 3684|
|2019-08-05 09:00:00|       264.0| 3680|
|2019-08-06 09:00:00|       264.0| 3674|
|2019-08-12 08:00:00|       264.0| 3671|
|2019-08-06 08:00:00|       264.0| 3650|
|2019-08-13 09:00:00|       264.0| 3649|
|2019-08-07 08:00:00|       264.0| 3647|
+-------------------+------------+-----+
only showing top

In [72]:
dfcount_2019_07.toPandas().to_csv('df_count_07.csv')

In [73]:
dfcount_2019_08.toPandas().to_csv('df_count_08.csv')